In [10]:
import pickle
import pandas as pd
import numpy as np

import sys
sys.path.append("C:/Users/blaze/Desktop/opsi_eeg_denoising/eeg-denoising/src")  # Adjust this path as needed
from metrics import rmse_value, pearson_correlation
from vis import plot_eeg

In [15]:
data_path = "C:/Users/blaze/Desktop/opsi_eeg_denoising/eeg-denoising/data/eeg_dataset.pkl"
with open(data_path, "rb") as f:
    eeg_datasets_dict = pickle.load(f)
#eeg_datasets_dict
results_emd = eeg_datasets_dict["results_emd_denoising"]

In [16]:
len(results_emd)

2

In [19]:
metrics_results = pd.DataFrame(columns=["IMFs removed", 'rmse', 'pearson_correlation'])
for i, dataset_results in enumerate(results_emd):
    signal_original = dataset_results['scaled_original_signal']
    parameter_sets = dataset_results['parameter_sets']
    denoised_signals = dataset_results['denoised_signals']
    print(f"Original signal shape: {len(parameter_sets)}")
    print(f"Parameter sets shape: {len(denoised_signals)}")

    rmse_ = np.mean([rmse_value(original, denoised) for original, denoised in zip(signal_original, denoised_signals)])
    pearson = np.mean([pearson_correlation(original, denoised) for original, denoised in zip(signal_original, denoised_signals)])
    print(f"Parameters: {parameter_sets[0]}, RMSE: {rmse_}, Pearson: {pearson}")

    metrics_results.loc[len(metrics_results)] = [parameter_sets[0], rmse_, pearson]
    print(f"Metrics results shape: {len(metrics_results)}")

Original signal shape: 64
Parameter sets shape: 64
Parameters: 1, RMSE: 0.3292508086232213, Pearson: 0.9348663856584899
Metrics results shape: 1
Original signal shape: 64
Parameter sets shape: 64
Parameters: 2, RMSE: 0.4684562985255226, Pearson: 0.8604543089103182
Metrics results shape: 2


In [20]:
len(denoised_signals)
    

64

In [6]:
print("Denoised: ", denoised_signals[0].size)
print("OG: ",signal_original[0].size)

Denoised:  60000
OG:  60000


In [ ]:
metrics_results

,IMFs removed,rmse,pearson_correlation
0,1.0,0.329251,0.934866
1,2.0,0.468456,0.860454


AttributeError: 'list' object has no attribute 'shape'

In [25]:
best_denoised_df = pd.DataFrame(denoised_signals[0].T, columns=dataset_results['dataset_original'].ch_names)
original_df = pd.DataFrame(signal_original[0].T, columns=dataset_results['dataset_original'].ch_names)
# best_denoised_df.head(5)
# len(dataset_results['dataset_original'].ch_names)

ValueError: Shape of passed values is (60000, 1), indices imply (60000, 64)